# Lab 1.6.2: Hyperparameter Optimization with Optuna

**Module:** 1.6 - Classical ML Foundations  
**Time:** 2 hours  
**Difficulty:** ⭐⭐⭐ (Intermediate)

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Understand why hyperparameter tuning matters
- [ ] Use Optuna for Bayesian hyperparameter optimization
- [ ] Define effective search spaces for XGBoost
- [ ] Visualize optimization history and parameter importance
- [ ] Apply cross-validation within hyperparameter search

---

## 📚 Prerequisites

- Completed: Lab 1.6.1 (Tabular Data Challenge)
- Knowledge of: XGBoost basics, cross-validation concept

---

## 🌍 Real-World Context

**The $100K Question:** How much accuracy can you squeeze out of hyperparameter tuning?

In Kaggle competitions, the difference between winning and losing is often in the hyperparameters:
- A 2019 Kaggle competition was won by improving XGBoost's AUC from 0.87 to 0.91 through tuning alone
- That 4% improvement represented a $100,000 prize difference!

In production:
- **Banks** tune credit scoring models to optimize profit/loss tradeoffs
- **E-commerce** tunes recommendation models for click-through rates
- **Healthcare** tunes diagnostic models for sensitivity/specificity balance

**Why Optuna?**
- Bayesian optimization (smarter than random search)
- Pruning (stops bad trials early)
- Visualization tools built-in
- State-of-the-art algorithms (TPE, CMA-ES)

---

## 🧒 ELI5: Hyperparameter Optimization

> **Imagine you're learning to bake the perfect chocolate chip cookie...**
>
> You have many "knobs" to adjust:
> - How much sugar? (sweetness)
> - What temperature? (crispiness)
> - How long to bake? (doneness)
> - How many chocolate chips? (richness)
>
> **Grid Search** = "I'll try EVERY combination"
> - 5 sugar levels × 5 temps × 5 times × 5 chip amounts = 625 batches! 😱
>
> **Random Search** = "I'll try random combinations"
> - Better than grid! Often finds good recipes faster
> - But still wastes time on obviously bad recipes
>
> **Bayesian Optimization (Optuna)** = "I'll learn from each batch!"
> - Batch 1: Too sweet → "Hmm, less sugar next time"
> - Batch 2: Perfect sweetness, but raw → "Keep sugar, increase temp"
> - Batch 3: Golden brown, delicious! → "Let's explore nearby settings"
>
> **In ML terms:** Optuna builds a probabilistic model of "which hyperparameters → good scores" and uses it to pick promising combinations to try next.

---

## Part 1: Setup and Data Preparation

Let's set up our environment and load the data.

In [ ]:
# Optuna is pre-installed in the NGC PyTorch container
# If running outside NGC, install with: pip install optuna plotly

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from time import time
import warnings
warnings.filterwarnings('ignore')

# Scikit-learn
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import mean_squared_error, r2_score

# XGBoost
import xgboost as xgb

# Optuna - The star of the show!
import optuna
from optuna.visualization import (
    plot_optimization_history,
    plot_param_importances,
    plot_contour,
    plot_parallel_coordinate
)

# Set random seed
np.random.seed(42)

# Plotting style with fallback for older matplotlib versions
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except OSError:
    try:
        plt.style.use('seaborn-whitegrid')
    except OSError:
        pass  # Use default style

print("✅ Libraries imported!")
print(f"📦 Optuna version: {optuna.__version__}")

In [ ]:
# Load California Housing dataset
print("📦 Loading California Housing Dataset...")
housing = fetch_california_housing()

X = housing.data
y = housing.target
feature_names = housing.feature_names

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\n📊 Dataset Info:")
print(f"  • Training samples: {len(X_train):,}")
print(f"  • Test samples: {len(X_test):,}")
print(f"  • Features: {len(feature_names)}")

## Part 2: Baseline Model (Default Hyperparameters)

First, let's establish a baseline with default XGBoost hyperparameters.

In [ ]:
# Train baseline model with defaults
print("🎯 Training Baseline XGBoost (Default Parameters)...")
print("=" * 60)

baseline_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    random_state=42,
    device='cuda',
    verbosity=0
)

# Cross-validation for robust estimate
cv_scores = cross_val_score(
    baseline_model, X_train, y_train,
    cv=5,
    scoring='neg_root_mean_squared_error'
)

baseline_rmse = -cv_scores.mean()
baseline_std = cv_scores.std()

# Also train on full training set for test evaluation
baseline_model.fit(X_train, y_train)
baseline_test_rmse = np.sqrt(mean_squared_error(y_test, baseline_model.predict(X_test)))

print(f"\n📊 Baseline Results (Default Hyperparameters):")
print(f"  • CV RMSE: ${baseline_rmse*100000:,.0f} (+/- ${baseline_std*100000:,.0f})")
print(f"  • Test RMSE: ${baseline_test_rmse*100000:,.0f}")
print(f"\n🎯 Our Goal: Beat ${baseline_rmse*100000:,.0f} CV RMSE through tuning!")

---

## Part 3: Understanding XGBoost Hyperparameters

Before we tune, let's understand what we're tuning!

### 🧒 ELI5: Key XGBoost Hyperparameters

> **Think of XGBoost as building a team of advisors...**
>
> | Parameter | Cookie Analogy | What It Does |
> |-----------|---------------|---------------|
> | `n_estimators` | Number of taste testers | More trees = more opinions, but slower |
> | `max_depth` | How picky each tester is | Deeper = more detailed, but may overthink |
> | `learning_rate` | How much to trust each opinion | Lower = need more testers to decide |
> | `min_child_weight` | Minimum confidence to speak up | Higher = only sure opinions count |
> | `subsample` | What % of cookies each tester tries | Less = more variety in opinions |
> | `colsample_bytree` | What % of criteria each tester checks | Less = each tester specializes |

In [ ]:
# XGBoost Hyperparameter Guide
print("📚 XGBoost Hyperparameter Guide")
print("=" * 70)

params_guide = """
┌─────────────────────────────────────────────────────────────────────┐
│                    XGBOOST HYPERPARAMETERS                          │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  🌲 TREE STRUCTURE                                                   │
│  ─────────────────                                                  │
│  max_depth (3-10)                                                   │
│    • Controls tree complexity                                       │
│    • Higher = more complex, risk of overfitting                     │
│    • Default: 6                                                     │
│                                                                      │
│  min_child_weight (1-10)                                            │
│    • Minimum sum of instance weight in a child                      │
│    • Higher = more conservative                                     │
│    • Default: 1                                                     │
│                                                                      │
│  gamma (0-5)                                                        │
│    • Minimum loss reduction to make a split                         │
│    • Higher = more conservative                                     │
│    • Default: 0                                                     │
│                                                                      │
│  📉 LEARNING RATE & TREES                                           │
│  ────────────────────────                                           │
│  learning_rate / eta (0.01-0.3)                                     │
│    • Step size shrinkage                                            │
│    • Lower = need more trees, but more robust                       │
│    • Default: 0.3                                                   │
│                                                                      │
│  n_estimators (100-1000)                                            │
│    • Number of boosting rounds                                      │
│    • More = better, but diminishing returns                         │
│    • Use early stopping!                                            │
│                                                                      │
│  🎲 SAMPLING                                                         │
│  ────────────                                                       │
│  subsample (0.5-1.0)                                                │
│    • Fraction of samples used per tree                              │
│    • Lower = more regularization                                    │
│    • Default: 1.0                                                   │
│                                                                      │
│  colsample_bytree (0.5-1.0)                                         │
│    • Fraction of features used per tree                             │
│    • Lower = more regularization                                    │
│    • Default: 1.0                                                   │
│                                                                      │
│  🔧 REGULARIZATION                                                   │
│  ───────────────────                                                │
│  reg_alpha (0-10)                                                   │
│    • L1 regularization                                              │
│    • Can drive feature weights to zero                              │
│                                                                      │
│  reg_lambda (0-10)                                                  │
│    • L2 regularization                                              │
│    • Shrinks weights, doesn't eliminate                             │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
"""
print(params_guide)

---

## Part 4: Optuna Hyperparameter Optimization

Now let's use Optuna's Bayesian optimization to find the best hyperparameters!

### How Optuna Works

1. **Define an objective function** that returns a score to minimize/maximize
2. **Create a study** that manages the optimization process
3. **Run trials** where Optuna suggests hyperparameters
4. **Optuna learns** which regions of the search space are promising

In [ ]:
# Define the objective function for Optuna
def objective(trial):
    """
    Optuna objective function for XGBoost hyperparameter tuning.
    
    This function:
    1. Suggests hyperparameters from defined search spaces
    2. Creates an XGBoost model with those parameters
    3. Evaluates using cross-validation
    4. Returns the score (RMSE to minimize)
    """
    
    # Define hyperparameter search space
    params = {
        # Tree structure
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.0, 5.0),
        
        # Learning
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        
        # Sampling
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        
        # Regularization
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        
        # Fixed parameters
        'objective': 'reg:squarederror',
        'device': 'cuda',
        'random_state': 42,
        'verbosity': 0
    }
    
    # Create model
    model = xgb.XGBRegressor(**params)
    
    # Cross-validation
    cv_scores = cross_val_score(
        model, X_train, y_train,
        cv=5,
        scoring='neg_root_mean_squared_error',
        n_jobs=-1
    )
    
    # Return mean RMSE (negated back to positive)
    return -cv_scores.mean()

print("✅ Objective function defined!")
print("\n📊 Search Space:")
print("  • max_depth: [3, 10]")
print("  • min_child_weight: [1, 10]")
print("  • gamma: [0.0, 5.0]")
print("  • learning_rate: [0.01, 0.3] (log scale)")
print("  • n_estimators: [100, 500]")
print("  • subsample: [0.5, 1.0]")
print("  • colsample_bytree: [0.5, 1.0]")
print("  • reg_alpha: [1e-8, 10.0] (log scale)")
print("  • reg_lambda: [1e-8, 10.0] (log scale)")

In [ ]:
# Create and run the Optuna study
print("🚀 Starting Optuna Hyperparameter Optimization...")
print("=" * 60)
print("This will run 100 trials with Bayesian optimization.")
print("Each trial uses 5-fold cross-validation.")
print("\n⏳ Estimated time: 2-5 minutes on GPU...")
print("=" * 60)

# Set up sampler (TPE = Tree-structured Parzen Estimator)
sampler = optuna.samplers.TPESampler(seed=42)

# Create study (minimize RMSE)
study = optuna.create_study(
    direction='minimize',
    sampler=sampler,
    study_name='xgboost_housing'
)

# Run optimization
start_time = time()

study.optimize(
    objective,
    n_trials=100,
    show_progress_bar=True,
    n_jobs=1  # Sequential for GPU
)

optimization_time = time() - start_time

print(f"\n✅ Optimization complete in {optimization_time:.1f} seconds!")

In [ ]:
# Display best results
print("🏆 Best Trial Results")
print("=" * 60)

best_trial = study.best_trial

print(f"\n📊 Best CV RMSE: ${best_trial.value*100000:,.0f}")
print(f"   Baseline RMSE: ${baseline_rmse*100000:,.0f}")
print(f"   Improvement: ${(baseline_rmse - best_trial.value)*100000:,.0f} ({(baseline_rmse - best_trial.value)/baseline_rmse*100:.1f}%)")

print(f"\n🔧 Best Hyperparameters:")
for key, value in best_trial.params.items():
    if isinstance(value, float):
        print(f"   • {key}: {value:.6f}")
    else:
        print(f"   • {key}: {value}")

---

## Part 5: Visualizing the Optimization Process

Optuna provides powerful visualization tools to understand the optimization process.

In [ ]:
# Optimization History
print("📈 Optimization History")
print("Shows how the best score improved over trials.")

fig = plot_optimization_history(study)
fig.update_layout(title='XGBoost Hyperparameter Optimization History')
fig.show()

In [ ]:
# Hyperparameter Importance
print("📊 Hyperparameter Importance")
print("Shows which hyperparameters had the biggest impact on performance.")

fig = plot_param_importances(study)
fig.update_layout(title='Which Hyperparameters Matter Most?')
fig.show()

In [ ]:
# Contour plot of top 2 parameters
print("🗺️ Contour Plot")
print("Shows interaction between the two most important hyperparameters.")

# Get top 2 important parameters
importances = optuna.importance.get_param_importances(study)
top_params = list(importances.keys())[:2]

fig = plot_contour(study, params=top_params)
fig.update_layout(title=f'Interaction: {top_params[0]} vs {top_params[1]}')
fig.show()

In [ ]:
# Parallel Coordinate Plot
print("📉 Parallel Coordinate Plot")
print("Shows relationships between all hyperparameters and the objective.")

fig = plot_parallel_coordinate(study)
fig.update_layout(title='Parallel Coordinate View of All Trials')
fig.show()

In [ ]:
# Create a summary visualization with matplotlib
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Optimization history
ax1 = axes[0, 0]
trial_numbers = [t.number for t in study.trials]
values = [t.value for t in study.trials]
best_values = [min(values[:i+1]) for i in range(len(values))]

ax1.scatter(trial_numbers, values, alpha=0.5, s=30, label='Trial RMSE')
ax1.plot(trial_numbers, best_values, 'r-', linewidth=2, label='Best so far')
ax1.axhline(baseline_rmse, color='green', linestyle='--', label='Baseline')
ax1.set_xlabel('Trial Number')
ax1.set_ylabel('CV RMSE')
ax1.set_title('Optimization History')
ax1.legend()

# 2. Parameter importance (bar chart)
ax2 = axes[0, 1]
importances = optuna.importance.get_param_importances(study)
params = list(importances.keys())
imp_values = list(importances.values())
colors = plt.cm.viridis(np.linspace(0, 0.8, len(params)))
ax2.barh(params, imp_values, color=colors)
ax2.set_xlabel('Importance')
ax2.set_title('Hyperparameter Importance')

# 3. Distribution of learning_rate values tried
ax3 = axes[1, 0]
lr_values = [t.params['learning_rate'] for t in study.trials]
rmse_values = [t.value for t in study.trials]
scatter = ax3.scatter(lr_values, rmse_values, c=trial_numbers, cmap='viridis', alpha=0.6)
ax3.set_xlabel('Learning Rate')
ax3.set_ylabel('CV RMSE')
ax3.set_title('Learning Rate vs Performance')
ax3.set_xscale('log')
plt.colorbar(scatter, ax=ax3, label='Trial #')

# 4. Distribution of max_depth values tried
ax4 = axes[1, 1]
depth_values = [t.params['max_depth'] for t in study.trials]
ax4.scatter(depth_values, rmse_values, c=trial_numbers, cmap='viridis', alpha=0.6)
ax4.set_xlabel('Max Depth')
ax4.set_ylabel('CV RMSE')
ax4.set_title('Max Depth vs Performance')

plt.tight_layout()
plt.savefig('optimization_summary.png', dpi=150, bbox_inches='tight')
plt.show()

print("💾 Saved optimization_summary.png")

---

## Part 6: Evaluate the Tuned Model

Now let's train the final model with the best hyperparameters and evaluate on the test set.

In [ ]:
# Train final model with best hyperparameters
print("🎯 Training Final Model with Best Hyperparameters...")
print("=" * 60)

# Get best params and add fixed params
best_params = best_trial.params.copy()
best_params['objective'] = 'reg:squarederror'
best_params['device'] = 'cuda'
best_params['random_state'] = 42
best_params['verbosity'] = 0

# Create final model
final_model = xgb.XGBRegressor(**best_params)

# Train on full training set
final_model.fit(X_train, y_train)

# Evaluate
train_pred = final_model.predict(X_train)
test_pred = final_model.predict(X_test)

train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, test_pred))
test_r2 = r2_score(y_test, test_pred)

print(f"\n📊 Final Model Performance:")
print(f"  • Train RMSE: ${train_rmse*100000:,.0f}")
print(f"  • Test RMSE:  ${test_rmse*100000:,.0f}")
print(f"  • Test R²:    {test_r2:.4f}")

print(f"\n🆚 Comparison with Baseline:")
print(f"  • Baseline Test RMSE: ${baseline_test_rmse*100000:,.0f}")
print(f"  • Tuned Test RMSE:    ${test_rmse*100000:,.0f}")
print(f"  • Improvement:        ${(baseline_test_rmse - test_rmse)*100000:,.0f} ({(baseline_test_rmse - test_rmse)/baseline_test_rmse*100:.1f}%)")

In [ ]:
# Feature importance from tuned model
print("📊 Feature Importance (Tuned Model)")
print("=" * 60)

# Get feature importance
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': final_model.feature_importances_
}).sort_values('Importance', ascending=True)

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.barh(importance_df['Feature'], importance_df['Importance'], 
               color='steelblue', alpha=0.8)
ax.set_xlabel('Feature Importance (Gain)')
ax.set_title('XGBoost Feature Importance (Tuned Model)')

# Add value labels
for bar, val in zip(bars, importance_df['Importance']):
    ax.text(val + 0.01, bar.get_y() + bar.get_height()/2, 
            f'{val:.3f}', va='center', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# Compare baseline vs tuned predictions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Baseline predictions
ax1 = axes[0]
baseline_pred = baseline_model.predict(X_test)
ax1.scatter(y_test, baseline_pred, alpha=0.5, s=20)
ax1.plot([0, 5], [0, 5], 'r--', linewidth=2)
ax1.set_xlabel('Actual Price ($100K)')
ax1.set_ylabel('Predicted Price ($100K)')
ax1.set_title(f'Baseline Model (RMSE: ${baseline_test_rmse*100000:,.0f})')
ax1.set_xlim(0, 5.5)
ax1.set_ylim(0, 5.5)

# Tuned predictions
ax2 = axes[1]
ax2.scatter(y_test, test_pred, alpha=0.5, s=20, color='green')
ax2.plot([0, 5], [0, 5], 'r--', linewidth=2)
ax2.set_xlabel('Actual Price ($100K)')
ax2.set_ylabel('Predicted Price ($100K)')
ax2.set_title(f'Tuned Model (RMSE: ${test_rmse*100000:,.0f})')
ax2.set_xlim(0, 5.5)
ax2.set_ylim(0, 5.5)

plt.tight_layout()
plt.show()

---

## Part 7: Best Practices and Tips

### Key Takeaways from Our Optimization

In [ ]:
# Summary of optimization insights
print("💡 Optimization Insights")
print("=" * 70)

# Get parameter importance
importances = optuna.importance.get_param_importances(study)

print("\n📊 Most Important Hyperparameters (in order):")
for i, (param, imp) in enumerate(importances.items(), 1):
    print(f"   {i}. {param}: {imp:.3f}")

print("\n🎯 Optimal Ranges Found:")
for param in list(importances.keys())[:3]:
    values = [t.params[param] for t in study.trials]
    best_val = best_trial.params[param]
    print(f"   • {param}: Best={best_val:.4f}, Range tried=[{min(values):.4f}, {max(values):.4f}]")

In [ ]:
# Best practices summary
print("📚 Best Practices for Hyperparameter Tuning")
print("=" * 70)

best_practices = """
┌─────────────────────────────────────────────────────────────────────┐
│                    HYPERPARAMETER TUNING BEST PRACTICES             │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  1️⃣  USE CROSS-VALIDATION                                            │
│     • Single train/val split can be misleading                      │
│     • 5-fold CV gives more robust estimates                         │
│     • Takes longer, but worth it!                                   │
│                                                                      │
│  2️⃣  USE LOG SCALE FOR CERTAIN PARAMETERS                            │
│     • learning_rate: [0.001, 0.3] should be log scale               │
│     • regularization: often spans orders of magnitude               │
│     • Log scale explores more efficiently                           │
│                                                                      │
│  3️⃣  START WITH WIDE RANGES, THEN NARROW                             │
│     • First run: explore wide ranges (100 trials)                   │
│     • Second run: narrow to promising regions                       │
│                                                                      │
│  4️⃣  USE PRUNING FOR EXPENSIVE EVALUATIONS                           │
│     • Optuna can stop bad trials early                              │
│     • Saves compute on obviously poor configurations                │
│                                                                      │
│  5️⃣  SAVE YOUR STUDIES                                               │
│     • Use optuna.create_study(storage='sqlite:///study.db')         │
│     • Resume optimization later                                     │
│     • Compare across experiments                                    │
│                                                                      │
│  6️⃣  DON'T OVERTUNE                                                  │
│     • Diminishing returns after ~100 trials                         │
│     • Risk of overfitting to validation set                         │
│     • Keep a hold-out test set untouched!                           │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
"""
print(best_practices)

---

## ✋ Try It Yourself

### Exercise 1: Tune with Pruning

Use Optuna's pruning feature to speed up optimization.

<details>
<summary>💡 Hint</summary>
Use `optuna.integration.XGBoostPruningCallback` with the XGBoost callback system.
</details>

### 💡 Advanced: Optuna Pruning with XGBoost

Optuna can **prune** (early stop) poorly performing trials to save time. This requires using XGBoost's native API instead of the sklearn-compatible API:

```python
# XGBoost has TWO APIs:
# 1. sklearn API (we've been using): xgb.XGBRegressor - easy, familiar
# 2. Native API: xgb.train() with DMatrix - more control, supports pruning

import xgboost as xgb

# Convert data to XGBoost's native format
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

# Native parameters (slightly different names)
params = {
    'objective': 'reg:squarederror',
    'max_depth': 6,
    'eta': 0.1,  # Called 'learning_rate' in sklearn API
}

# Train with native API
bst = xgb.train(
    params,
    dtrain,
    num_boost_round=500,
    evals=[(dval, 'validation')],
    early_stopping_rounds=10,
    verbose_eval=False
)

# Predict
preds = bst.predict(dval)
```

**Pruning callback:**
```python
from optuna.integration import XGBoostPruningCallback

# Inside your objective function:
pruning_callback = XGBoostPruningCallback(trial, 'validation-rmse')
bst = xgb.train(params, dtrain, callbacks=[pruning_callback], ...)
```

In [ ]:
# Exercise 1: Your code here
# Implement pruning to stop bad trials early

# def objective_with_pruning(trial):
#     params = { ... }  # Same as before
#     
#     # Add pruning callback
#     pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'validation-rmse')
#     
#     # Use XGBoost's native API with callback
#     dtrain = xgb.DMatrix(X_train, label=y_train)
#     dval = xgb.DMatrix(X_val, label=y_val)
#     
#     bst = xgb.train(
#         params,
#         dtrain,
#         num_boost_round=500,
#         evals=[(dval, 'validation')],
#         callbacks=[pruning_callback],
#         verbose_eval=False
#     )
#     
#     return ...

### Exercise 2: Tune LightGBM

Apply the same optimization technique to LightGBM.

<details>
<summary>💡 Hint</summary>
LightGBM has similar parameters but different names:
- `num_leaves` instead of focusing on `max_depth`
- `min_child_samples` instead of `min_child_weight`
</details>

### 💡 LightGBM Introduction

LightGBM is another gradient boosting library, often faster than XGBoost:

```python
# Install: pip install lightgbm (or use NGC container)
import lightgbm as lgb

# LightGBM uses slightly different parameter names:
model = lgb.LGBMRegressor(
    num_leaves=31,          # Max leaves per tree (replaces max_depth focus)
    min_child_samples=20,   # Similar to min_child_weight
    learning_rate=0.1,
    n_estimators=100,
    verbosity=-1            # Suppress output
)

model.fit(X_train, y_train)
predictions = model.predict(X_test)
```

**Key differences from XGBoost:**
- `num_leaves` controls complexity (instead of primarily `max_depth`)
- Leaf-wise tree growth (vs level-wise in XGBoost)
- Often faster for large datasets
- `min_child_samples` instead of `min_child_weight`

In [ ]:
# Exercise 2: Your code here
# Tune LightGBM and compare with XGBoost

# import lightgbm as lgb
# 
# def lgb_objective(trial):
#     params = {
#         'num_leaves': trial.suggest_int('num_leaves', 20, 300),
#         'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
#         'n_estimators': trial.suggest_int('n_estimators', 100, 500),
#         ...
#     }
#     ...

### Exercise 3: Multi-Objective Optimization

Optimize for both accuracy AND training speed.

<details>
<summary>💡 Hint</summary>
Use `optuna.create_study(directions=['minimize', 'minimize'])` for multi-objective.
Return a tuple: (rmse, training_time)
</details>

### 💡 Multi-Objective Optimization with Optuna

Sometimes you want to optimize for multiple objectives (e.g., accuracy AND speed):

```python
# Create a multi-objective study
study = optuna.create_study(
    directions=['minimize', 'minimize']  # Both objectives to minimize
    # Use ['minimize', 'maximize'] if second objective should be maximized
)

# Objective function returns a TUPLE of values
def multi_objective(trial):
    params = {...}
    
    start = time()
    model.fit(X_train, y_train)
    train_time = time() - start
    
    rmse = compute_rmse(model, X_val, y_val)
    
    return rmse, train_time  # Return tuple of objectives

study.optimize(multi_objective, n_trials=50)

# Get Pareto front (best trade-offs)
pareto_trials = study.best_trials  # Note: best_trials (plural) for multi-objective
```

**Key differences from single-objective:**
- Use `directions` (plural) with a list
- Return a tuple from objective function
- Use `study.best_trials` to get Pareto-optimal solutions

In [ ]:
# Exercise 3: Your code here
# Multi-objective: minimize both RMSE and training time

# def multi_objective(trial):
#     params = { ... }
#     
#     start = time()
#     # ... train model ...
#     train_time = time() - start
#     
#     return rmse, train_time  # Return tuple
#
# study = optuna.create_study(directions=['minimize', 'minimize'])
# study.optimize(multi_objective, n_trials=50)

---

## ⚠️ Common Mistakes

### Mistake 1: Not Using Cross-Validation

In [ ]:
# ❌ Wrong: Single train/val split
# def bad_objective(trial):
#     model = xgb.XGBRegressor(**params)
#     model.fit(X_train_split, y_train_split)  # Fixed split!
#     return model.score(X_val_split, y_val_split)  # Overfits to this split

# ✅ Right: Use cross-validation
print("💡 Always use cross-validation in hyperparameter tuning!")
print("   A single split can give misleading results.")
print("   CV gives a more robust estimate of generalization.")

### Mistake 2: Searching Linear Scale for Learning Rate

In [ ]:
# ❌ Wrong: Linear scale for learning rate
# 'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3)
# This spends 90% of samples between 0.1-0.3, ignoring 0.01-0.1!

# ✅ Right: Log scale
# 'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True)

print("💡 Use log scale for parameters that span orders of magnitude!")
print("   • learning_rate: 0.001 to 0.3 → log scale")
print("   • reg_alpha/reg_lambda: 1e-8 to 10 → log scale")
print("   • max_depth: 3 to 10 → linear scale (small range)")

### Mistake 3: Using Test Set During Tuning

In [ ]:
# ❌ Wrong: Using test set to select hyperparameters
# def bad_objective(trial):
#     ...
#     return model.score(X_TEST, y_TEST)  # NEVER DO THIS!

# ✅ Right: Keep test set completely separate
print("⚠️ CRITICAL: Never use the test set during hyperparameter tuning!")
print("")
print("   The test set must be completely untouched until final evaluation.")
print("   Otherwise, you're overfitting to the test set!")
print("")
print("   Correct workflow:")
print("   1. Split data: train/val/test")
print("   2. Use CV on train+val for tuning")
print("   3. Train final model on train+val")
print("   4. Evaluate ONCE on test")

---

## 🎉 Checkpoint

Congratulations! You've mastered hyperparameter optimization with Optuna. You've learned:

- ✅ **Optuna basics**: Studies, trials, and objectives
- ✅ **Search space design**: Integer, float, categorical, and log scale
- ✅ **XGBoost parameters**: What they do and how to tune them
- ✅ **Visualization**: Understanding optimization history and parameter importance
- ✅ **Best practices**: Cross-validation, log scale, and avoiding test set leakage

---

## 🚀 Challenge (Optional)

**The Kaggle Challenge:** Can you beat the leaderboard?

1. Download a Kaggle tabular competition dataset
2. Use Optuna to tune XGBoost, LightGBM, and CatBoost
3. Ensemble the best models
4. Submit to Kaggle and compare with the leaderboard!

Good starter competitions:
- House Prices: Advanced Regression Techniques
- Titanic: Machine Learning from Disaster
- Spaceship Titanic

---

## 📖 Further Reading

- [Optuna Documentation](https://optuna.readthedocs.io/)
- [Optuna Tutorial](https://optuna.readthedocs.io/en/stable/tutorial/index.html)
- [XGBoost Parameter Tuning Guide](https://xgboost.readthedocs.io/en/latest/tutorials/param_tuning.html)
- [Bayesian Optimization Primer](https://distill.pub/2020/bayesian-optimization/)

---

## 🧹 Cleanup

In [ ]:
# Clean up
import gc

# Delete models
del baseline_model, final_model
del study

# Garbage collection
gc.collect()

print("✅ Cleanup complete!")

---

## ➡️ Next Steps

Continue to **Lab 1.6.3: RAPIDS Acceleration** to learn how to speed up classical ML by 10-100x using GPU acceleration with cuML!